In [5]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import itertools
import random

# ========== 固定隨機種子 ==========
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

# ========== 讀取與處理資料 ==========
red = pd.read_csv("./winequality-red_train.csv")
red['type'] = 0
white = pd.read_csv("./winequality-white_train.csv")
white['type'] = 1
train = pd.concat([red, white], ignore_index=True)

X = train.drop('quality', axis=1).values
y = train['quality'].values

scaler = StandardScaler()
X = scaler.fit_transform(X)

# ========== 資料分割 ==========
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# ========== 模型定義 ==========
class WineQualityNN(nn.Module):
    def __init__(self, input_dim, num_layers, dropout, use_bn):
        super(WineQualityNN, self).__init__()
        dims = [input_dim, 512, 256, 128, 64, 32, 16]
        layers = []
        for i in range(num_layers):
            layers.append(nn.Linear(dims[i], dims[i + 1]))
            if use_bn:
                layers.append(nn.BatchNorm1d(dims[i + 1]))
            layers.append(nn.ReLU())
            if dropout:
                layers.append(nn.Dropout(0.3))
        self.hidden = nn.Sequential(*layers)
        self.output = nn.Linear(dims[num_layers], 11)

    def forward(self, x):
        return self.output(self.hidden(x))

# ========== 訓練與驗證 ==========
def train_and_validate(params):
    model = WineQualityNN(input_dim=X.shape[1],
                          num_layers=params['num_layers'],
                          dropout=params['dropout'],
                          use_bn=params['use_bn'])
    optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'])
    criterion = nn.CrossEntropyLoss()

    train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                                  torch.tensor(y_train, dtype=torch.long))
    val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32),
                                torch.tensor(y_val, dtype=torch.long))
    train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=params['batch_size'], shuffle=False)

    for epoch in range(20):
        model.train()
        for xb, yb in train_loader:
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()

    # 驗證準確率
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            preds = model(xb)
            predicted = preds.argmax(1)
            correct += (predicted == yb).sum().item()
            total += yb.size(0)
    return correct / total

# ========== 超參數組合 ==========
param_grid = {
    'num_layers': [3, 4],
    'dropout': [True, False],
    'use_bn': [True, False],
    'batch_size': [32, 64],
    'lr': [0.001, 0.003, 0.005],
    'weight_decay': [1e-4, 1e-5]
}

param_names = list(param_grid.keys())
param_combinations = list(itertools.product(*param_grid.values()))

# ========== 搜尋前五組最佳參數 ==========
results = []
for i, values in enumerate(param_combinations):
    set_seed(42)
    params = dict(zip(param_names, values))
    acc = train_and_validate(params)
    results.append((acc, params))
    print(f"[{i+1}/{len(param_combinations)}] Accuracy: {acc:.4f} | Params: {params}")

# 排序並取前五
results.sort(key=lambda x: x[0], reverse=True)
print("\n 前五名最佳參數組合:")
for rank, (acc, params) in enumerate(results[:5], 1):
    print(f"Top {rank}: Accuracy = {acc:.4f} | Params = {params}")


[1/96] Accuracy: 0.5663 | Params: {'num_layers': 3, 'dropout': True, 'use_bn': True, 'batch_size': 32, 'lr': 0.001, 'weight_decay': 0.0001}
[2/96] Accuracy: 0.5615 | Params: {'num_layers': 3, 'dropout': True, 'use_bn': True, 'batch_size': 32, 'lr': 0.001, 'weight_decay': 1e-05}
[3/96] Accuracy: 0.5529 | Params: {'num_layers': 3, 'dropout': True, 'use_bn': True, 'batch_size': 32, 'lr': 0.003, 'weight_decay': 0.0001}
[4/96] Accuracy: 0.5683 | Params: {'num_layers': 3, 'dropout': True, 'use_bn': True, 'batch_size': 32, 'lr': 0.003, 'weight_decay': 1e-05}
[5/96] Accuracy: 0.5423 | Params: {'num_layers': 3, 'dropout': True, 'use_bn': True, 'batch_size': 32, 'lr': 0.005, 'weight_decay': 0.0001}
[6/96] Accuracy: 0.5654 | Params: {'num_layers': 3, 'dropout': True, 'use_bn': True, 'batch_size': 32, 'lr': 0.005, 'weight_decay': 1e-05}
[7/96] Accuracy: 0.5625 | Params: {'num_layers': 3, 'dropout': True, 'use_bn': True, 'batch_size': 64, 'lr': 0.001, 'weight_decay': 0.0001}
[8/96] Accuracy: 0.5740

KeyboardInterrupt: 